# Train a Model to Trade on the Stock Market

![](https://www.xm.com/wp-content/uploads/2017/09/Stock-Charts-1.jpg)

## Efficient Market Hypothesis

Future prices cannot be predicted by analyzing prices from the past. Excess returns cannot be earned in the long run by using investment strategies based on historical share prices or other historical data. Technical analysis techniques will not be able to consistently produce excess returns.

 

## Technical Analysis


In finance, technical analysis is an analysis methodology for forecasting the direction of prices through the study of past market data, primarily price and volume. The efficacy of technical  analysis is disputed by the efficient-market hypothesis which states that stock market prices are essentially unpredictable.

 

## Can a neural net learn Technical Analysis techniques to predict the market and make good trades?

Are we about to get rich?

In [1]:
import random
from indicators import *
from transform import *
import pandas_profiling
import pandas_datareader as web
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import h2o
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
import cufflinks as cf
from plots import *
import numpy as np
import operator

# Suppress unwatned warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

# Set some defaults for how pandas displays
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Enter These Variables

In [2]:
stockSymbol = "MSFT"
market = "NASDAQ"
invest = 10000

start_date = pd.datetime(2013,1,1)
split_validation = pd.datetime(2018,1,1)
split_test = pd.datetime(2018,4,1)

### Let's use a machine learning library called H2O

In [3]:
#h2o.cluster().shutdown()

In [4]:
h2o.init(max_mem_size="76G")
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_162"; OpenJDK Runtime Environment (build 1.8.0_162-8u162-b12-0ubuntu0.16.04.2-b12); OpenJDK 64-Bit Server VM (build 25.162-b12, mixed mode)
  Starting server from /home/olmsteadbrett/.local/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpPf72Aq
  JVM stdout: /tmp/tmpPf72Aq/h2o_olmsteadbrett_started_from_python.out
  JVM stderr: /tmp/tmpPf72Aq/h2o_olmsteadbrett_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,06 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.7
H2O cluster version age:,12 days
H2O cluster name:,H2O_from_python_olmsteadbrett_i81cul
H2O cluster total nodes:,1
H2O cluster free memory:,67.6 Gb
H2O cluster total cores:,96
H2O cluster allowed cores:,96
H2O cluster status:,"accepting new members, healthy"


### Connect to Morningstar and Download the Historical Data  

In [5]:
df = web.DataReader(stockSymbol, 'morningstar', start=start_date)
df = df.reset_index(drop=False)
df = df.drop('Symbol', axis=1)
df.set_index('Date', inplace=True)
df = df[['Open','High', 'Low', 'Close','Volume']]


### Plot.ly is a Sweet Charting Library

In [6]:
myPlotlyKey = 'conh5EnFad0Z9Lz6mVWr'
py.sign_in(username='bretto777',api_key=myPlotlyKey)

fig = simpleStockChart(df,stockSymbol,split_validation,split_test)
py.iplot(fig, validate=False)

The draw time for this plot will be slow for clients without much RAM.


### Technical Analysis
Add technical analysis indicators used by the pros! Hopefully our model will learn how these indicators can predict tomorrow's price movements.

In [7]:
%%capture
df.reset_index(drop=False, inplace=True, col_level=0)

typical_price(df, high_col = 'High', low_col = 'Low', close_col = 'Close')
money_flow_index(df, vol_col='Volume')
rsi(df, close_col='Close')
ema(df, period=26, column='Close')
ema(df, period=12, column='Close')
macd(df, period_long=26, period_short=12, period_signal=9, column='Close')
on_balance_volume(df, trend_periods=21, close_col='Close', vol_col='Volume')
price_volume_trend(df, trend_periods=21, close_col='Close', vol_col='Volume')
bollinger_bands(df, trend_periods=20, close_col='Close')

In [8]:
fig = technicalAnalysisChart(df)
py.iplot(fig, validate=False)

In [9]:
%%capture
windowTransform(df)

In [10]:
classifyDF = df.loc[:,['Close Tomorrow s','Close','Close Change','Close Change s',
                       'Close-1','Close-1 Change','Close-1 s',
                       'Close-2','Close-2 Change','Close-2 s',
                       'Close-3','Close-3 Change','Close-3 s',
                       'Close-4','Close-4 Change','Close-4 s',
                       'Close-5','Close-5 Change','Close-5 s',
                       'Close-6','Close-6 Change','Close-6 s',
                       'Close-7','Close-7 Change','Close-7 s',
                       'Close-8','Close-8 Change','Close-8 s',
                       'Close-9','Close-9 Change','Close-9 s',
                       'Close-10','Close-10 Change','Close-10 s',
                       'Close-11','Close-11 Change','Close-11 s',                       
                       'Close-12','Close-12 Change','Close-12 s',
                       'Close-13','Close-13 Change','Close-13 s',
                       'Close-14','Close-14 Change','Close-14 s',
                       'Close-15','Close-15 Change','Close-15 s',
                       'Close-16','Close-16 Change','Close-16 s',
                       'Close-17','Close-17 Change','Close-17 s',
                       'Close-18','Close-18 Change','Close-18 s',
                       'Close-19','Close-19 Change','Close-19 s',
                       'Close-20','Close-20 Change','Close-20 s',
                       'Close-21','Close-21 Change','Close-21 s',
                       'Close-22','Close-22 Change','Close-22 s',
                       'Close-23',
                       'rsi','rsi-1','rsi Change',
                       'macd_signal_line-1','macd_signal_line-2','macd_signal_line-3','macd_signal_line-4','macd_signal_line-5','macd_signal_line-6','macd_signal_line-7',
                       'macd_val-1','macd_val-2','macd_val-3','macd_val-4','macd_val-5','macd_val-6','macd_val-7',
                       'money_flow_index','money_flow_index-1','money_flow_index Change',
                       'Volume','Volume-1',
                       'Date',
                       'DayOfWeek','DayOfYear','WeekOfYear',
                       'High','High-1',
                       'Low','Low-1',
                       'Open','Open-1']]

In [11]:
trainingDF = classifyDF.loc[classifyDF['Date'] <= split_validation]
validationDF = classifyDF[classifyDF['Date'].isin(pd.date_range(split_validation,split_test))]
testDF = classifyDF.loc[classifyDF['Date'] > split_test]

trainingDF = trainingDF.dropna(axis=0, how='any')
validationDF = validationDF.dropna(axis=0, how='any')
testDF = testDF.fillna(value=0)

trainingDF.set_index('Date', inplace=True)
validationDF.set_index('Date', inplace=True)
testDF.set_index('Date', inplace=True)

In [12]:
#scaler = MinMaxScaler()
#trainingDF = pd.DataFrame(scaler.fit_transform(trainingDF), columns=trainingDF.columns)
#trainingDF['Close Tomorrow s'] = trainingDF['Close Tomorrow s'].astype('int')
#validationDF = pd.DataFrame(scaler.fit_transform(validationDF), columns=validationDF.columns)
#validationDF['Close Tomorrow s'] = validationDF['Close Tomorrow s'].astype('int')
#testDF = pd.DataFrame(scaler.fit_transform(testDF), columns=testDF.columns)
#testDF['Close Tomorrow s'] = testDF['Close Tomorrow s'].astype('int')

In [13]:
train = h2o.H2OFrame(trainingDF, destination_frame="train.hex")
valid = h2o.H2OFrame(validationDF, destination_frame="valid.hex")
test = h2o.H2OFrame(testDF, destination_frame="test.hex")

train['Close Tomorrow s'] = train['Close Tomorrow s'].asfactor()
valid['Close Tomorrow s'] = valid['Close Tomorrow s'].asfactor()
test['Close Tomorrow s'] = test['Close Tomorrow s'].asfactor()

# Set predictor and response variables
y = "Close Tomorrow s"
x = train.columns
x.remove(y)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


### Train the Deep Neural Net
Grid search for a good model

In [15]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.grid.grid_search import H2OGridSearch

hidden_opt = [[25,25,25,25,25],
              [50,75,75,50],
              [75,100,100,75],
              [50,300,300,50],
              [10,10,100,10,10],
              [1000,1000],
              [50,50,50]]
l1_opt = [0.001,0.0001,0.00001]
l2_opt = [0.001,0.0001]
rate_opt= [0.01,0.001,0.0001]
activation_opt=["Rectifier","TanhWithDropout"]
epochs_opt=[3500,2500]

hyper_parameters = {"hidden":hidden_opt, 
                    "l1":l1_opt, 
                    "l2":l2_opt,
                    "rate":rate_opt, 
                    "activation":activation_opt,
                    "epochs":epochs_opt}

search_criteria = {"strategy":"RandomDiscrete", "max_models":10, "seed":123456}



model_grid = H2OGridSearch(H2ODeepLearningEstimator(),
                           hyper_params=hyper_parameters,
                           search_criteria=search_criteria,
                           grid_id='test8')
model_grid.train(x=x, 
                 y=y,                                   
                 fast_mode=True,                   
                 stopping_rounds=10, 
                 stopping_tolerance=0,
                 stopping_metric='mean_per_class_error',
                 adaptive_rate=True,                
                 training_frame=train, 
                 validation_frame=valid,
                 score_interval=10)

deeplearning Grid Build progress: |███████████████████████████████████████| 100%


In [104]:
model_gridperf1 = model_grid.get_grid(sort_by='mean_per_class_error', decreasing=False)
bestModel = model_gridperf1[0]          
h2o.save_model(model=bestModel, path="/home/olmsteadbrett/")


u'/home/olmsteadbrett/test8_model_1'

In [15]:
bestModel = h2o.load_model(path="/home/olmsteadbrett/stocks/test8_model_1")

In [16]:
importances = bestModel.varimp(use_pandas=True)
importances = importances.loc[:,['variable','relative_importance']].groupby('variable').mean()
importances.sort_values(by="relative_importance", ascending=False).iplot(kind='bar', colors='#5AC4F2', theme='white')

In [17]:
cm = bestModel.confusion_matrix(valid)
cm = cm.as_data_frame()
confusionMatrix = FF.create_table(cm)
confusionMatrix.layout.height=300
confusionMatrix.layout.width=800
confusionMatrix.layout.font.size=17
py.iplot(confusionMatrix)

In [18]:
pBestModel = pd.concat([bestModel.predict(test_data=test).as_data_frame(use_pandas=True).reset_index(drop=True), testDF.reset_index(drop=False)], axis =1)


deeplearning prediction progress: |███████████████████████████████████████| 100%


In [58]:
prices = df.loc[df['Date'] > split_test]
prices = prices.loc[:,['Date','Close','Close Tomorrow','Close Tomorrow s']]
profit = pBestModel.loc[:,['predict']]
profit = pd.concat([prices.reset_index(drop=True), profit.reset_index(drop=True)], axis =1)
profit['predict'] = profit['predict'].astype(np.float64)

profit['profit'] = invest

profit['Best Possible'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['Close Tomorrow s'] > 0 else 0, axis=1)

profit['Randomly Invest'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if random.randint(1,1000) > random.randint(1,1000) else 0, axis=1)
profit['profit'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['predict'] > 0 else 0, axis=1)

#profit.profit.sum()
#profit.groupby('predict').sum() 
print "Profit               : $" + str(profit.profit.sum())
print "Random investment    : $" + str(profit['Randomly Invest'].sum())
print "Best possible outcome: $" + str(profit['Best Possible'].sum())

profit



Profit               : $675.2043828431179
Random investment    : $-133.21170536246427
Best possible outcome: $1516.9113359091752


,Date,Close,Close Tomorrow,Close Tomorrow s,predict,profit,Best Possible,Randomly Invest
0,2018-04-02,88.52,89.71,1.0,1.0,134.432897,134.432897,134.432897
1,2018-04-03,89.71,92.33,1.0,1.0,292.052168,292.052168,0.000000
2,2018-04-04,92.33,92.38,1.0,-1.0,0.000000,5.415358,0.000000
3,2018-04-05,92.38,90.23,-1.0,1.0,-232.734358,0.000000,-232.734358
4,2018-04-06,90.23,90.77,1.0,1.0,59.847058,59.847058,59.847058
5,2018-04-09,90.77,92.88,1.0,1.0,232.455657,232.455657,232.455657
6,2018-04-10,92.88,91.86,-1.0,-1.0,0.000000,0.000000,-109.819121
7,2018-04-11,91.86,93.58,1.0,1.0,187.241454,187.241454,0.000000
8,2018-04-12,93.58,93.08,-1.0,-1.0,0.000000,0.000000,-53.430220
9,2018-04-13,93.08,94.17,1.0,-1.0,0.000000,117.103567,0.000000
